In [ ]:
# imports

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import averaging
importlib.reload(averaging)
from averaging import average_outer_ring
from averaging import average_rings

In [ ]:
# basic tests

# initializations
size = 40
radius = 10
a = np.zeros((size,size))

# get circular mask
axis = np.arange(a.shape[0])
center = int(len(axis)/2)-0.5
mask = (axis[np.newaxis,:]-center)**2 + (axis[:,np.newaxis]-center)**2 < radius**2

# set values inside mask to 1 and plot
a[mask] = 1
plt.imshow(a)

In [ ]:
# basic tests for multiple rings

size = 40
radii = [5, 10, 15]

axis = np.arange(size)
center = int(len(axis)/2)-0.5
rings = {}
dist = (axis[np.newaxis,:]-center)**2 + (axis[:,np.newaxis]-center)**2
for i in range(len(radii)):
    mask = (dist > radii[i]**2)
    if i<len(radii)-1: mask = ((mask) & (dist < radii[i+1]**2))
    rings[radii[i]] = mask
    
for ring in rings.values():
    plt.figure()
    plt.imshow(ring)

In [ ]:
# get some example histograms

fname = '../data/examples/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1.parquet'
df = pd.read_parquet(fname)
xbins = df['Xbins'][0]
ybins = df['Ybins'][0]
nhists = 5
hists = np.array([df['histo'][i].reshape(xbins,ybins) for i in range(nhists)])
runs = np.array([df['fromrun'][i] for i in range(nhists)])
lumis = np.array([df['fromlumi'][i] for i in range(nhists)])
print('Shape of hists array: {}'.format(hists.shape))
print('Runs: {}'.format(runs))
print('Lumis: {}'.format(lumis))

In [ ]:
# plot raw data

for i, hist in enumerate(hists):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# make average of outer rings and plot

idx = 1
hist = hists[idx]
hist = average_outer_ring(hist, hists, radius=50)
fig, ax = plot_histogram(hist)
ax.text(0.02, 1.02, 'Run: {}, lumi: {}, outer ring averaged'.format(runs[idx], lumis[idx]), transform=ax.transAxes, fontsize=12)

In [ ]:
# make averages of outer ring with fixed refrence histograms

hists_averaged = average_outer_ring(hists, hists, radius=50)
for i, hist in enumerate(hists_averaged):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, outer ring averaged'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# make averages of outer ring with sliding reference histograms

hists_averaged = average_outer_ring(hists, 2, radius=50)
for i, hist in enumerate(hists_averaged):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, outer ring averaged'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# make averages of inner and outer ring with sliding reference histograms

radii = {0:2, 50: 5}
hists_averaged = average_rings(hists, radii)
for i, hist in enumerate(hists_averaged):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, averaged'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)